# 集合Collections

有效的整理 TensorFlow程序中的资源也是计算图的一个重要功能。在一个计算图中，可以通过集合(`collection`)来管理不同类别的资源。比如，`tf.add_to_collection`函可以将资源加入一个或多个集合中，然后通过`tf.get_collection`获取一个集合里面的所有资源。这里的资源可以是张量、变量或者运行 TensorFlow 所需要的队列资源。

## `tf.get_collection()`

函数原型：`tf.get_collection(key, scope=Name)`。

使用默认的计算图，对`Graph.get_collection()`的包装。以列表的形式返回指定 `collection` 里的 `values`.

参数：

1. key：collection的 key。例如，GraphKeys class 包含标准的 collections 的 names。
2. scope：可选项。

## `tf.add_to_collection()`

函数原型：`tf.add_to_collection()`

使用默认的计算图，对 `Graph.add_to_collection` 的包装。把 `value` 添加到 `collections（name）`。

## 常用的集合

1. tf.GraphKeys.VARIABLES: 所有变量的集合.
2. tf.GraphKeys.GLOBAL_VARIABLES: 变量的默认集合。由 `Variable` 和 `get_variable()` 产生的变量将自动存入此集合。一般，`TRAINABLE_VARIABLES` 是 `MODEL_VARIABLES` 的子集，所有的 `MODEL_VARIABLES` 是 `GLOBAL_VARIABLES` 的子集。
3. tf.GraphKeys.LOCAL_VARIABLES: 本地变量的子集。通常是临时变量，例如：计数器，使用`tf.contrib.framwork.local_variable`加入此集合.
4. tf.GraphKeys.MODEL_VARIABLES: 变量对象的子集。通常用于前向传播的模型中。使用`tf.contrib.framwork.local_variable`加入此集合.
5. tf.GraphKeys.TRAINABLE_VARIABLES: 可学习的变量(一般指神经网络中的参数)，模型训练、生成模型可视化内容.
6. tf.GraphKeys.SUMMARIES: 日志生成相关的张量,TensorFlow计算可视化.
7. tf.GraphKeys.QUEUE_RUNNERS: 处理输入的QueueRunner, 输入处理.
8. tf.GraphKeys.MOVING_AVERAGE_VARIABLES: 所有计算了滑动平均值的变量, 计算变量的滑动平均值.
9. tf.GraphKeys.REGULARIZATION_LOSSES: 在计算图结构中收集的损失集合, 计算模型损失.

## 举例说明

In [10]:
import tensorflow as tf

In [11]:
tf.reset_default_graph()

# 定义一个常量
W = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name="const_W")

# 定义三个正则化变量
lamda = 1.0
L1 = tf.contrib.layers.l1_regularizer(lamda)(W)
L2 = tf.contrib.layers.l2_regularizer(lamda)(W)
L1_L2 = tf.contrib.layers.l1_l2_regularizer(lamda)(W)

# 把正则变量加入"regularizer_loss1"、"regularizer_loss2" 和 损失函数集合
tf.add_to_collection("regularizer_loss", L1)
tf.add_to_collection("regularizer_loss", L2)
tf.add_to_collection("regularizer_loss", L1_L2)
tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, L1)
tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, L2)
tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, L1_L2)

# 显示“regularizer_loss1”、“regularizer_loss2”和标准损失函数集合内的数据
# 返回 Tensor 的列表
loss1 = tf.get_collection("regularizer_loss")
loss2 = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
print("loss1: {}".format(loss1))
print("loss2: {}".format(loss2))

with tf.Session() as sess:
    print("loss1: {}".format(sess.run(loss1)))
    print("loss2: {}".format(sess.run(loss2)))

loss1: [<tf.Tensor 'l1_regularizer:0' shape=() dtype=float32>, <tf.Tensor 'l2_regularizer:0' shape=() dtype=float32>, <tf.Tensor 'l1_l2_regularizer:0' shape=() dtype=float32>]
loss2: [<tf.Tensor 'l1_regularizer:0' shape=() dtype=float32>, <tf.Tensor 'l2_regularizer:0' shape=() dtype=float32>, <tf.Tensor 'l1_l2_regularizer:0' shape=() dtype=float32>]
loss1: [21.0, 45.5, 66.5]
loss2: [21.0, 45.5, 66.5]
